In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(3, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.PS)
node[2] = Queue(model, 'Queue2', SchedStrategy.FCFS)

jobclass = np.empty(2, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 1, node[0], 0)
jobclass[1] = ClosedClass(model, 'Class2', 2, node[0], 0)

# renewal
map21 = APH([1,0],[[-2,2],[0,-0.5]])
map22 = MAP([-1],[1]) 
# non-renewal
map31 = MAP([[-20,0],[0,-1]],[[0,20],[0.8,0.2]])
map32 = MAP([[-20,0],[0,-1]],[[0,20],[0.8,0.2]])

node[0].set_service(jobclass[0], HyperExp.fit_mean_and_scv(1.0,25.0))
node[1].set_service(jobclass[0], map21)
node[2].set_service(jobclass[0], map31)

node[0].set_service(jobclass[1], HyperExp.fit_mean_and_scv(1.0,25.0))
node[1].set_service(jobclass[1], map22)
node[2].set_service(jobclass[1], map32)

model.add_link(node[0], node[0])
model.add_link(node[0], node[1])
model.add_link(node[0], node[2])
model.add_link(node[1], node[0])
model.add_link(node[2], node[0])

node[0].set_routing(jobclass[0],RoutingStrategy.RROBIN) # if changed to RAND the EJML exception goes
node[1].set_prob_routing(jobclass[0], node[0], 1.0)
node[2].set_prob_routing(jobclass[0], node[0], 1.0)

node[0].set_routing(jobclass[1],RoutingStrategy.RROBIN) # if changed to RAND the EJML exception goes
node[1].set_prob_routing(jobclass[1], node[0], 1.0)
node[2].set_prob_routing(jobclass[1], node[0], 1.0)

In [3]:
solver = np.array([], dtype=object)
solver = np.append(solver, JMT(model, seed=23000))
solver = np.append(solver, CTMC(model, keep=False))

In [4]:
AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].get_name()}')
    try:
        AvgTable[s] = solver[s].avg_table()
        print(AvgTable[s])
    except Exception as e:
        print(f"Solver error: {type(e).__name__} - Non-FCFS MAP stations may not be supported")


SOLVER: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.3988 0.3988 1.0105  1.0105 0.3979 0.3979
  Delay   Class2 1.1190 1.1190 1.0005  1.0005 1.1312 1.1312
 Queue1   Class1 0.4980 0.3281 3.7827  3.7827 0.1314 0.1314
 Queue1   Class2 0.5848 0.3747 1.5819  1.5819 0.3820 0.3820
 Queue2   Class1 0.1033 0.0756 0.7971  0.7971 0.1314 0.1314
 Queue2   Class2 0.2818 0.2342 0.6790  0.6790 0.3815 0.3815
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   Class1  0.3988  0.3988  1.0105  1.0105  0.3979  0.3979
1   Delay   Class2  1.1190  1.1190  1.0005  1.0005  1.1312  1.1312
2  Queue1   Class1  0.4980  0.3281  3.7827  3.7827  0.1314  0.1314
3  Queue1   Class2  0.5848  0.3747  1.5819  1.5819  0.3820  0.3820
4  Queue2   Class1  0.1033  0.0756  0.7971  0.7971  0.1314  0.1314
5  Queue2   Class2  0.2818  0.2342  0.6790  0.6790  0.3815  0.3815

SOLVER: CTMC
Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.1430 0.1430 1.0000